In [1]:
import pandas as pd 
import numpy as np 

In [27]:
import math
import numpy as np
import pandas as pd

# --- Step 1: Load and Prepare Data ---
df = pd.read_csv(r"D:\02_PYTHON-CODES\17_NUMERICAL_COMPUTING_2\ndvi-2010-2020.csv")
df['date'] = pd.to_datetime(df['date'], format="%Y_%m_%d")
df['t'] = range(len(df))

# --- Step 2: Select subset for interpolation ---
x = df['t'].values[:50]       # first 10 time points
y = df['1'].values[:50]       # NDVI values
h = x[1] - x[0]
n = len(y)

# --- Step 3: Build difference table ---
diff_table = np.zeros((n, n))
diff_table[:, 0] = y
for i in range(1, n):
    for j in range(n - i):
        diff_table[j][i] = diff_table[j + 1][i - 1] - diff_table[j][i - 1]

# --- Step 4: Interpolation Formulas ---
def newton_forward(x, y, x0, h, xp):
    p = (xp - x0) / h
    yp = y[0]
    p_term = 1
    for i in range(1, n):
        p_term *= (p - (i - 1))
        yp += (p_term / math.factorial(i)) * diff_table[0][i]
    return yp

def newton_backward(x, y, xn, h, xp):
    p = (xp - xn) / h
    yp = y[-1]
    p_term = 1
    for i in range(1, n):
        p_term *= (p + (i - 1))
        yp += (p_term / math.factorial(i)) * diff_table[n - i - 1][i]
    return yp

def gauss_forward(x, y, x_mid, h, xp):
    m = len(y) // 2
    p = (xp - x[m]) / h
    yp = y[m]
    for i in range(1, n - 1):
        idx = m - (i // 2)
        p_term = 1
        for j in range(i):
            p_term *= (p - j + (0.5 if i % 2 == 0 else -0.5))
        if 0 <= idx < n - i:
            yp += (p_term / math.factorial(i)) * diff_table[idx][i]
    return yp

def gauss_backward(x, y, x_mid, h, xp):
    m = len(y) // 2
    p = (xp - x[m]) / h
    yp = y[m]
    for i in range(1, n - 1):
        idx = m - (i // 2)
        p_term = 1
        for j in range(i):
            p_term *= (p + j - (0.5 if i % 2 == 0 else -0.5))
        if 0 <= idx < n - i:
            yp += (p_term / math.factorial(i)) * diff_table[idx][i]
    return yp

def stirling_central(x, y, x_mid, h, xp):
    m = len(y) // 2
    p = (xp - x[m]) / h
    yp = y[m]
    i = 1
    while (2 * i) < n and (m - i) >= 0:
        # Avoid out of range access
        term1 = (p / math.factorial(2 * i - 1)) * (
            diff_table[m - i][2 * i - 1] + diff_table[m - i + 1][2 * i - 1]
        ) / 2
        term2 = ((p * p - i * i) / math.factorial(2 * i)) * diff_table[m - i][2 * i]
        yp += term1 + term2
        i += 1
    return yp


# --- Step 5: Choose interpolation point ---
xp = 25  # between t=5 and t=6

# --- Step 6: Apply all methods ---
nf = newton_forward(x, y, x0=x[0], h=h, xp=xp)
nb = newton_backward(x, y, xn=x[-1], h=h, xp=xp)
gf = gauss_forward(x, y, x_mid=x[len(x)//2], h=h, xp=xp)
gb = gauss_backward(x, y, x_mid=x[len(x)//2], h=h, xp=xp)
st = stirling_central(x, y, x_mid=x[len(x)//2], h=h, xp=xp)

# --- Step 7: Print Results ---
print(f"\nInterpolated NDVI value at x = {xp}")
print(f"Newton Forward  : {nf:.5f}")
print(f"Newton Backward : {nb:.5f}")
print(f"Gauss Forward   : {gf:.5f}")
print(f"Gauss Backward  : {gb:.5f}")
print(f"Stirling Method : {st:.5f}")



Interpolated NDVI value at x = 25
Newton Forward  : 0.44900
Newton Backward : 0.44900
Gauss Forward   : 52665707155.25715
Gauss Backward  : -50094888932.76791
Stirling Method : 0.43613


In [28]:
print(df[['t', '1']].head())


   t      1
0  0  0.493
1  1  0.469
2  2  0.436
3  3  0.401
4  4  0.379
